<h1 style="color: #6485DB;">Problem Statement</h1>
<hr>

### Overview
This is a regression problem where you need to predict the average spend of customers for the next 3 months.

### Data Description
**train.csv**<br>
It contains the training data with advertisement details as described in the last section.

**test.csv**<br>
It has details of the customer for which we have to predict the spend for the next 3 months.

**Data Dictionary**<br>
The Data Dictionary is mentioned in detail in the file data_dictionary.xlsx.
It contains information about the features of the dataset.

<hr>

## 1. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder , StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error as msle, r2_score

## Data Description
Let's first look at the data description.

In [ ]:
data = pd.read_excel("data/Data_Dictionary.xlsx", index_col = 0)
data

## 2. Load the data

In [ ]:
df = pd.read_csv('data/TRAIN.csv')
test = pd.read_csv('data/TEST.csv')

In [ ]:
df

In [ ]:
# Check the shape of data
df.shape

In [ ]:
# Describe the data to check central tendancies
df.describe()

**Insight:**<br>
We can notice that **Age column has some outliers** so we'll treat those values.

In [ ]:
# Let's check the information about all the columns
df.info()

## 3. Treating NULL Values

In [ ]:
# Let's check for NULL values

df.isnull().sum()

**Insights:**<br>
1. Columns **`personal_loan_active`, `vehicle_loan_active`, `personal_loan_closed`, `vehicle_loan_closed`, `investment_1`, `investment_2`, `investment_3`, `investment_4`, `loan_enq`** have more than 90% null values, so we'll remove these features.

2. For other columns with Null values, we will treat those with respective **'median'** values.

In [ ]:
# Dropping unnecessary columns

df.drop(columns=['personal_loan_active', 'vehicle_loan_active' ,'personal_loan_closed', 'vehicle_loan_closed','investment_1','investment_2','investment_3','investment_4','loan_enq'],axis=1,inplace=True)
test.drop(columns=['personal_loan_active', 'vehicle_loan_active' ,'personal_loan_closed', 'vehicle_loan_closed','investment_1','investment_2','investment_3','investment_4','loan_enq'],axis=1,inplace=True)

df.isnull().sum() # Checking for more columns with NULL values

In [ ]:
# For the remaining columns with NULL values, we can impute them with respective median values

df.fillna(df.median(), inplace=True)

df.isnull().sum()

In [ ]:
test.fillna(test.median(), inplace=True)

test.isnull().sum()

## 4. Outliers treatment

In [ ]:
# Check the outliers in 'Age' column using boxplot
plt.figure(figsize=(4,6))
sns.boxplot(y=df['age'])
plt.show()

In [ ]:
# As noticed in the boxplot, all values above 75 can be considered as outliers
# So let's treat them with the median of 'Age' column

df['age'] = np.where(df['age']>75, df['age'].median(), df['age'])
test['age'] = np.where(test['age']>75, test['age'].median(), test['age'])

In [ ]:
# Let's check again for outliers in 'age' col.
df['age'].describe()

In [ ]:
test['age'].describe()

In [ ]:
# There might be many columns with outliers, let's treat them at once.

# --- Train data ---
def cap_data(data):
    for col in data.columns:
        if (((data[col].dtype)=='float64') | ((data[col].dtype)=='int64')):
            percentiles = data[col].quantile([0.1,0.9]).values
            data[col][data[col] <= percentiles[0]] = percentiles[0]
            data[col][data[col] >= percentiles[1]] = percentiles[1]
        else:
            data[col]=data[col]
    return data


df = cap_data(df)
test = cap_data(test)

In [ ]:
df.describe()

## 5. Exploratory Data Analysis

In [ ]:
# Gender Ratio
print(df['gender'].value_counts(normalize=True)*100)

df['gender'].value_counts().plot.bar(cmap='summer')
plt.show()

**Insight:**<br>
- The Male account holders are way too many than Females. With 85.7% and 14.3% respectively. 

In [ ]:
# Gender vs Age Ratio
plt.figure(figsize=(9,7))
sns.countplot(df['age'],hue=df['gender'],data=df)
plt.title("Gender vs Age Ratio", fontsize=20)
plt.xticks(rotation=65)
plt.show()

**Insight:**<br>
- The gender column is Right Skewed.
- Although there are a lot of Male counts than Females in each Age range, it seems that **there is a decrease in customer counts with increase in age.**
- Pension age of 61 tends to have more customers than age range 50-60

In [ ]:
df.shape, test.shape

In [ ]:
# Save the 'id' column from Test data
ids = test['id']
ids

In [ ]:
# Dividing the data
X = df.drop(columns=['id','cc_cons','region_code'], axis=1)
test = test.drop(columns=['id','region_code'], axis=1)

y = df['cc_cons']

X.shape, test.shape

In [ ]:
# Let's check the correlation between each column
plt.figure(figsize=(30, 20))
cols_corr = df.corr().abs()
mask = np.zeros_like(cols_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(cols_corr, linewidth=.02, vmin=0, vmax=1, cmap='summer', annot=True, mask = mask)
plt.xticks(rotation=75)
plt.show()

**Insight:**<br>
- There are certain columns which have much higher correlation b/w them, We will have to eliminate the collinearity.

In [ ]:
upper = cols_corr.where(np.triu(np.ones(cols_corr.shape), k=1).astype(np.bool)) # selecting upper triangle of corr_matrix

# Getting index of columns with correlation greater than 0.65 to drop them out
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
to_drop

In [ ]:
df = df.drop(columns = to_drop, axis=1)

df.shape

In [ ]:
# Function to generate RMSLE score

def rmsle(y_test, y_pred):
    
    sum=0.0
    
    for x,y in zip(y_test, y_pred):
        
        if x < 0 or y < 0:
            continue
        
        p = np.log(y+1)
        r = np.log(x+1)
        sum += (p - r)**2
    
    return (sum / len(y_pred)) ** 0.5

## 6. Label Encoding
Let's encode the categorical variables using LabelEncoder()

In [ ]:
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print(cat_cols)

In [ ]:
le = LabelEncoder()
for i in cat_cols:
    X[i] = le.fit_transform(X[i]) # train data
    test[i] = le.fit_transform(test[i]) # test data

X

## 7. Standard Scaling
Let's scale the numerical variables using StandardScaler()

In [ ]:
num_cols = [i for i in X.columns if i not in cat_cols]
print(num_cols)

In [ ]:
ss = StandardScaler()
X[num_cols] = ss.fit_transform(X[num_cols])
test[num_cols] = ss.fit_transform(test[num_cols])

X.describe()

## 8. Train Test Split
Let's split our training data into train and validation data needed to build our model

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y,test_size=0.3,random_state=12)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

## 9. Model Building

In [ ]:
linreg = LinearRegression()
rfreg = RandomForestRegressor()
cbreg = CatBoostRegressor(iterations=100,
                          learning_rate=0.1,
                          eval_metric='MSLE')

In [ ]:
# Fitting the models

linreg.fit(x_train, y_train)
rfreg.fit(x_train, y_train)
cbreg.fit(x_train, y_train)

In [ ]:
# Make Predictions

y_pred_linreg = linreg.predict(x_val)
y_pred_rfreg = rfreg.predict(x_val)
y_pred_cbreg = cbreg.predict(x_val)

In [ ]:
# Check for RMSLE scores
print("RMSLE Scores:\n")
print('LinearRegression:\t', (msle(y_val, y_pred_linreg)*0.5)*100)
print('RandomForestRegressor:\t', (msle(y_val, y_pred_rfreg)*0.5)*100)
print('CatBoostRegressor:\t', (msle(y_val, y_pred_cbreg)*0.5)*100)

## 10. Make Predictions

In [ ]:
# Using Linear Regression
final_pred = linreg.predict(test)

In [ ]:
submission = pd.DataFrame()
submission['id'] = ids.astype(int)
submission['cc_cons'] = final_pred
submission.to_csv('submission_linreg.csv', index=False, header=True)

In [ ]:
final_pred2 = cbreg.predict(test)

submission2 = pd.DataFrame()
submission2['id'] = ids.astype(int)
submission2['cc_cons'] = final_pred2
submission2.to_csv('submission_cbreg.csv', index=False, header=True)